# AltumAge

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import tensorflow as tf
from tensorflow.keras.models import load_model

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'altumage'
data_type = 'methylation'
model_class = 'AltumAge'
species = 'Homo sapiens'
year = 2022
approved_by_author = '✅'
citation = "de Lima Camillo, Lucas Paulo, Louis R. Lapierre, and Ritambhara Singh. \"A pan-tissue DNA-methylation epigenetic clock based on deep learning.\" npj Aging 8.1 (2022): 4."
doi = 'https://doi.org/10.1038/s41514-022-00085-y'
notes = None

## Download necessary data

#### Download GitHub repository

In [3]:
github_url = "https://github.com/rsinghlab/AltumAge.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

##  Load data

In [4]:
AltumAge = load_model('AltumAge/example_dependencies/AltumAge.h5')  # Load your trained TensorFlow model
features = pd.read_pickle('AltumAge/example_dependencies/multi_platform_cpgs.pkl').tolist()
scaler = pd.read_pickle('AltumAge/example_dependencies/scaler.pkl')

## Extract features and weights

First, let's extract the features and weights:

In [5]:
weights = {}
for layer in AltumAge.layers:
    weights[layer.name] = layer.get_weights()

## Load weights into pyaging model

#### AltumAge

In [6]:
model = pya.models.AltumAge()

# Function to copy weights from TensorFlow to PyTorch
def copy_weights(torch_layer, tf_weights, bn=False):
    with torch.no_grad():
        if bn:
            torch_layer.weight.data = torch.tensor(tf_weights[0]).float()
            torch_layer.bias.data = torch.tensor(tf_weights[1]).float()
            torch_layer.running_mean.data = torch.tensor(tf_weights[2]).float()
            torch_layer.running_var.data = torch.tensor(tf_weights[3]).float()
        else:
            torch_layer.weight.data = torch.tensor(tf_weights[0]).T.float()
            torch_layer.bias.data = torch.tensor(tf_weights[1]).float()

# Now copy the weights
copy_weights(model.bn1, weights['batch_normalization_84'], bn=True)
copy_weights(model.linear1, weights['dense_84'])
copy_weights(model.bn2, weights['batch_normalization_85'], bn=True)
copy_weights(model.linear2, weights['dense_85'])
copy_weights(model.bn3, weights['batch_normalization_86'], bn=True)
copy_weights(model.linear3, weights['dense_86'])
copy_weights(model.bn4, weights['batch_normalization_87'], bn=True)
copy_weights(model.linear4, weights['dense_87'])
copy_weights(model.bn5, weights['batch_normalization_88'], bn=True)
copy_weights(model.linear5, weights['dense_88'])
copy_weights(model.bn6, weights['batch_normalization_89'], bn=True)
copy_weights(model.linear6, weights['dense_89'])

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [7]:
reference_feature_values = dict(zip(features, scaler.center_))

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format:

In [8]:
def preprocessing_function(x):
    scaler = preprocessing_helper_objects[0]
    x_scaled = scaler.transform(x)
    return x_scaled
preprocessing_function_string = marshal.dumps(preprocessing_function.__code__)

preprocessing_helper_objects = [scaler]

preprocessing = {
    'name': 'scale',
    'preprocessing_function': preprocessing_function_string,
    'preprocessing_helper_objects': preprocessing_helper_objects
}

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [9]:
postprocessing = None

## Check all data objects
<a id="8"></a>

Let's print all data objects to check if they make sense.

#### features

In [12]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [10]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [11]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [14]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [13]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary
<a id="9"></a>

Let's put everything together and save:

In [15]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory
<a id="10"></a>

Delete all files that are not clock jupyter notebooks:

In [16]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: AltumAge
Deleted folder: .ipynb_checkpoints
